# statsperform API + bq backloading

In [ ]:
import subprocess
import json
import pandas as pd
import os

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)


offered_feeds = pd.read_csv('statsperform_feeds.csv')

{
    k:v for k, v in zip(offered_feeds.FeedCode, offered_feeds.FeedWidgetName)
}

In [ ]:
def _access_statsperform_api(feed_name: str,
                             tourney_cal_id: str = None,
                             match_id: str = None
                             ):

    proxy_url = "http://127.0.0.1:3128"
    statsperform_base_url = 'https://api.performfeeds.com/soccerdata'
    auth_key = str(os.environ['STATSPERFORM_API_KEY'])

    master_dict = {
        '_rt':'b',
        '_fmt':'json'
    }

    if feed_name == 'match':
        assert tourney_cal_id is not None, "To access match feed data, a tournament calendar ID must be passed in."
        master_dict['tmcl'] = tourney_cal_id
        master_dict['_pgSz'] = 1000
    if feed_name == 'matchstats':
        assert match_id is not None, "To access match stats feed, a match ID must be passed in."
        master_dict['fx'] = match_id
        master_dict['detailed'] = 'yes'
        master_dict['people'] = 'yes'

    query_string = '&'.join([f'{k}={v}' for k, v in master_dict.items()])

    q_command = f""" curl -x "{proxy_url}" '{statsperform_base_url}/{feed_name}/{auth_key}/authorized?&{query_string}' """ if feed_name == 'tournamentcalendar' else \
        f""" curl -x "{proxy_url}" '{statsperform_base_url}/{feed_name}/{auth_key}?&{query_string}' """

    # print(q_command)

    process = subprocess.run(
                q_command,
                shell=True,
                capture_output=True,
                check=False # Set to True if you want a CalledProcessError for non-zero exit codes
            )

    stdout = process.stdout
    json_output = json.loads(stdout)
    return json_output


## tournaments

In [ ]:

def get_statsperform_tourneys(to_bq=False):
    # Get all available tournament calendar IDs with OT2 feed
    feed = 'tournamentcalendar'
    comps = _access_statsperform_api(feed)
    competitions = pd.DataFrame(comps['competition'])
    competitions = competitions.explode('tournamentCalendar')

    extended_tourneys = pd.json_normalize(competitions['tournamentCalendar'])
    extended_tourneys.columns = ['tourneyCalId', 'includesVenues', 'tcOcId', 'tcName', 'startDate', 'endDate', 'active', 'lastUpdated', 'includesStandings']

    final_df = pd.concat([competitions.drop(columns=['tournamentCalendar']).reset_index(drop=True),
                      extended_tourneys.reset_index(drop=True)], axis=1)
                                
    if to_bq:
        try:
            final_df.to_gbq('soccer_simulations.tourneycal_data',
                    'prizepicksanalytics',
                    if_exists='fail')
        except Exception as e:
            print("Table already exists on BQ!")

    return final_df

## matches

In [ ]:
# Get all matches with MA1 feed
def _get_all_matches_in_tourneycal(tourney_cal_id: str):

    all_matches = _access_statsperform_api(feed_name='match',
                         tourney_cal_id=tourney_cal_id)

    return pd.DataFrame([x['matchInfo'] for x in all_matches['match']])

## process data

In [ ]:
matches = _get_all_matches_in_tourneycal(tourney_cal_id)

In [ ]:
from tqdm.notebook import tqdm_notebook
import config
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
teams, players = backload_season_data(tourney_cal_id)

In [ ]:

# getting access to player data using MA2 feed
def backload_season_data(tourney_cal_id, 
                         game_limit=None,
                         project_name: str = 'prizepicksanalytics',
                        table_name: str = f'soccer_simulations.schema_match_data'):
    
    matches = _get_all_matches_in_tourneycal(tourney_cal_id)

    processed_match_ids = list(get_processed_match_ids().match_id)
    list_of_matches = [x for x in matches.id.unique() if x not in processed_match_ids]

    if game_limit:
        list_of_matches = list_of_matches[:game_limit]

    all_teams = []
    all_players = []

    for i, id in enumerate(list_of_matches):

        if id in processed_match_ids:
            continue

        print(f"Now processing match #{i}: {id}")
        team, player = process_game_data(id)

        all_players.append(player)
        all_teams.append(team)
        

    if all_teams and all_players:
        team_data_to_bq = pd.concat(all_teams, axis=0, ignore_index=True)
        team_data_to_bq.to_gbq(table_name.replace('schema', 'team'),
                project_name,
                if_exists='append')
        
        player_data_to_bq = pd.concat(all_players, axis=0, ignore_index=True)
        player_data_to_bq.to_gbq(table_name.replace('schema', 'player'),
                project_name,
                if_exists='append')
        
        print(f"Team and Player Data Uploaded for the following Game Ids: {team_data_to_bq.match_id.unique()}")
        return team, player
    else:
        return None, None

In [ ]:
# players.to_csv('./test_players.csv', index=False)
# teams.to_csv('./test_teams.csv', index=False)
# players = pd.read_csv('./test_players.csv', index_col=0)
# teams = pd.read_csv('./test_teams.csv', index_col=0)

In [ ]:
from google.cloud import bigquery as bqc


# DataFrame should be of only one match
def check_bq_duplicates(dataframe: pd.DataFrame,
                        schema: str,
                        project_name: str = 'prizepicksanalytics',
                        table_name: str = f'soccer_simulations.schema_match_data'):
    
    assert dataframe.shape[0] != 0, "Empty dataframes not accepted"
    assert schema in ['team', 'player'], "Only team and player data accepted"
    table_name = table_name.replace('schema', schema)

    match_ids = list(dataframe.match_id.unique())

    column_key = 'team_id' if schema == 'team' else 'playerId'

    query = f"""
        select match_id, {column_key}
        from {project_name}.{table_name}
        where match_id in ({", ".join(["'" + x + "'" for x in match_ids])}) 
    """

    rows_on_bq = execute_bq_query(query)

    if rows_on_bq.shape[0] == dataframe.shape[0]:
        print(f"Data already loaded onto BQ for Match Id: {match_ids} Schema: {schema}")
        return
    else:
        dataframe = dataframe.merge(rows_on_bq,
                                    on=['match_id', column_key],
                                    how='left',
                                    indicator=True)
        
        dataframe = dataframe[dataframe['_merge'] == 'left_only'].drop(columns=['_merge'])
        

    return dataframe

def get_processed_match_ids():
    query = """
        select distinct match_id
        from prizepicksanalytics.soccer_simulations.team_match_data
    """

    return execute_bq_query(query)


def execute_bq_query(query: str) -> pd.DataFrame:
    bigquery_client = bqc.Client(project='prizepicksanalytics')
    query_results = bigquery_client.query(query)
    data = query_results.to_dataframe()
    bigquery_client.close()

    return data



In [ ]:
def process_game_data(match_id):
    
    match_data = _access_statsperform_api(feed_name='matchstats',
                         match_id=match_id)
    
    # TODO: 
    team_data = _aggregate_team_data(match_data)

    player_data = _aggregate_player_data(match_data)
    
    return team_data, player_data


In [ ]:
# method to aggregate team data

def _aggregate_team_data(match_stats):

    # collect tournament calendar, competition and team information
    tourney_cal_id, tourney_cal_season = match_stats['matchInfo']['tournamentCalendar']['id'], \
        match_stats['matchInfo']['tournamentCalendar']['name']
    competition_id, competition_name = match_stats['matchInfo']['competition']['id'], \
        match_stats['matchInfo']['competition']['name']
    
    competitors = match_stats['matchInfo']['contestant']
    
    # turn match data into dataframe
    match_data = pd.DataFrame(match_stats['liveData']['lineUp'])

    match_list = []

    for _, team in match_data.iterrows():
        # turn statistics into numerics and get team object from competitor
        team_stats = pd.DataFrame(team.stat)
        team_stats['value'] = pd.to_numeric(team_stats.value)
        competitor = competitors[0] if competitors[0]['id'] == team.contestantId else competitors[1]

        # transpose statistics to make ts dataframe (team statistics)
        ts = team_stats[['type', 'value']].set_index('type').T

        stat_cols = ts.columns
        missing_cols = set(config.all_team_match_stats).difference(stat_cols)

        if missing_cols:
            missing_df = pd.DataFrame(0, index=ts.index, columns=list(missing_cols), dtype=float)
            ts = pd.concat([ts, missing_df], axis=1)

        # create dataframe for cs (config statistics)
        cs = pd.DataFrame({
            'competition_id': [competition_id],
            'competition_name': [competition_name],
            'tourney_cal_id': [tourney_cal_id],
            'tourney_cal_name': [tourney_cal_season],
            'match_id' : [match_stats['matchInfo']['id']],
            'match_date': match_stats['matchInfo']['date'],
            'team_id' : [str(competitor['id'])],
            'team_name': [str(competitor['shortName'])],
            'home' : True if competitor['position'] == 'home' else False,
        })

        team_row = pd.concat([cs.reset_index(drop=True), ts[config.all_team_match_stats].reset_index(drop=True)], axis=1)
        team_row['formationUsed'] = str(team.formationUsed)

        match_list.append(team_row)

    # return dataframe of len == 2 with rows representing both teams, sharing the same game_id
    return pd.concat(match_list).fillna(0)

In [ ]:
# method to aggregate player data

def _aggregate_player_data(match_request):
    match_data = pd.DataFrame(match_request['liveData']['lineUp'])

    subs = pd.DataFrame(match_request['liveData']['substitute']).set_index('playerOnId').to_dict('index')

    list_of_dfs = []

    for _, team in match_data.iterrows():
        players = pd.DataFrame(team['player'])

        # replace substitution position with boolean column
        players['isSub'] = players.position.apply(lambda x: True if x == 'Substitute' else False)
        cols_to_keep = ['playerId', 'matchName', 'position', 'positionSide', 'isSub']

        # turn statistics json into pandas dataframe + make all numbers into floats
        stats = players.stat.apply(lambda lisa: {x['type']:float(x['value']) for x in lisa})
        players = pd.concat([players[cols_to_keep].reset_index(drop=True), pd.json_normalize(stats).reset_index(drop=True)], axis=1)

        # remove all players from dataframe who didn't play
        players = players[~players['minsPlayed'].isna()]

        # checks all subs in players dataframe, replace values with subbed out player info
        subbed_players = players[players['isSub'] == True]

        # replace substitution position info with player who subbed out for substitute
        for i, row in subbed_players.iterrows():
            sub_info = subs[row.playerId]
            subbed_out_player = players[players['playerId'] == sub_info['playerOffId']].iloc[0]

            players.at[i, 'position'] =  subbed_out_player.position
            players.at[i, 'positionSide'] =  subbed_out_player.positionSide
            players.at[i, 'formationPlace'] =  subbed_out_player.formationPlace

        # add config columns
        players.insert(0, 'match_id', match_request['matchInfo']['id'])
        players.insert(1, 'team_id', team.contestantId)
        config_cols = ['match_id', 'team_id'] + cols_to_keep

        # add all columns not found in player data to keep StatsPerform detailed player statistics schema 
        # (schema can be found in config.py)
        stat_cols = players.drop(config_cols, axis=1).columns
        missing_cols = set(config.all_player_match_stats).difference(stat_cols)

        if missing_cols:
            missing_df = pd.DataFrame(0, index=players.index, columns=list(missing_cols), dtype=float)
            players = pd.concat([players, missing_df], axis=1)

        list_of_dfs.append(players[config_cols + config.all_player_match_stats])

    # return dataframe of all players with their statistics
    return pd.concat(list_of_dfs, axis=0, ignore_index=True).fillna(0)
    

# feature engineering

In [103]:
from data import *

tourneys = get_statsperform_tourneys()

In [ ]:
ligue12425 = 'a7htj8rtzib7a2xx7b3xs04d0'



,tourneyCalId,includesVenues,tcOcId,tcName,startDate,endDate,active,lastUpdated,includesStandings
0,1z35p4iuhfxxdfaqjwzkqn2fo,yes,24546,2025 Morocco,2025-12-21Z,2026-01-18Z,yes,2025-06-09T09:40:41Z,yes
1,d8grid5nhzleybw2n5cpccxsk,yes,21250,2023 Côte d'Ivoire,2024-01-13Z,2024-02-11Z,no,2025-06-09T09:40:41Z,yes
2,3ny27xml9skko03pycmbrv52i,yes,17985,2021 Cameroon,2022-01-09Z,2022-02-06Z,no,2025-06-09T09:40:41Z,yes
3,2ip4f1aefabczfkw80hj7uz8p,yes,13580,2019 Egypt,2019-06-21Z,2019-07-19Z,no,2025-06-09T09:40:41Z,yes
4,3l4bzc8syz1ea2dnv453kp89g,yes,26158,2025,2025-01-23Z,2025-12-14Z,yes,2025-06-06T01:27:07Z,yes
5,534vuyemq3e4biv283g33zf2s,yes,24459,2024,2024-01-25Z,2024-12-17Z,no,2025-06-06T01:27:07Z,yes
6,5jc60w3y7cps00lavutor9zbo,yes,22664,2023,2023-01-27Z,2023-12-17Z,no,2025-06-06T01:27:07Z,yes
7,aefcj9288gsf8elnhzleg9dsk,yes,21099,2022,2022-02-10Z,2022-10-26Z,no,2025-06-06T01:27:07Z,yes
8,9pqtmpr3w8jm73y0eb8hmum8k,yes,26113,2025,2025-03-29Z,2025-12-21Z,yes,2025-06-05T01:02:10Z,yes
9,a2yu8vfo8wha3vza31s2o8zkk,yes,24419,2024,2024-04-13Z,2024-12-08Z,no,2025-06-02T22:09:55Z,yes
